In [20]:
from preprocessing import preprocessing
import numpy as np
from pyscipopt import Model, quicksum
from itertools import product
from tabulate import tabulate

courses, rooms = preprocessing()
print(courses)
print(rooms)

[('MAT-108 - A01', 'Siebert, Oliver', 35, 'WELLMAN 230', 'MWF 0210-0300 PM')
 ('MAT-108 - A02', 'Siebert, Oliver', 35, 'WELLMAN 230', 'MWF 0210-0300 PM')
 ('MAT-108 - B01', 'Liu, Fu', 35, 'OLSON 250', 'MWF 0110-0200 PM')
 ('MAT-108 - B02', 'Liu, Fu', 35, 'OLSON 250', 'MWF 0110-0200 PM')
 ('MAT-108 - D01', 'Liu, Fu', 35, 'OLSON 223', 'MWF 1100-1150 AM')
 ('MAT-108 - D02', 'Liu, Fu', 35, 'OLSON 223', 'MWF 1100-1150 AM')
 ('MAT-115A - A01', 'Soshnikov, Alexander', 30, 'HOAGLD 168', 'MWF 1210-0100 PM')
 ('MAT-115A - A02', 'Soshnikov, Alexander', 30, 'HOAGLD 168', 'MWF 1210-0100 PM')
 ('MAT-115A - B01', 'Li, Junxian', 30, 'HOAGLD 168', 'MWF 0310-0400 PM')
 ('MAT-115A - B02', 'Li, Junxian', 30, 'HOAGLD 168', 'MWF 0310-0400 PM')
 ('MAT-118A - 001', 'Jacob, Adam', 60, 'WELLMN 212', 'MWF 0210-0300 PM')
 ('MAT-119A - 001', 'Chaudhuri, Rishidev', 70, 'OLSON 106', 'MWF 0900-0950 AM')
 ('MAT-127A - A01', 'Iyer, Sameer', 30, 'WALKER 1320', 'MWF 0900-0950 AM')
 ('MAT-127A - A02', 'Iyer, Sameer', 30, 

In [21]:
m = courses.shape[0]
n = rooms.shape[0]
conflict_cap_array = np.zeros((m, n))
time = np.array([i for i in range(8, 18)])
conflict_instructor_list = []

for i, c in enumerate(courses):
    for j, r in enumerate(rooms):
        if c[2] > r[1]:
            conflict_cap_array[i, j] = 0
        else:
            conflict_cap_array[i, j] = 1

np.set_printoptions(threshold=np.inf)

conflict_cap_array = conflict_cap_array.astype(int)

for i in range(m):
    for j in range(i + 1, m):
        if courses[i][1] == courses[j][1]:
            conflict_instructor_list.append((i, j))    

In [22]:
def solve(courses: np.ndarray, rooms: np.ndarray, time: np.ndarray, M: np.ndarray, L: list, optimal=True, ILP=True):
    vtype = "B" if ILP else "C"
    C = courses.shape[0]
    R = rooms.shape[0]
    T = time.shape[0]
    model = Model()
    x = {}
    y = {}
    valid_indices = np.argwhere(M == 1)
    for c, r in valid_indices:
        for t in range(T):
            x[(c, r, t)] = model.addVar(name=f"x_{c}_{r}_{t}", vtype=vtype, lb=0, ub=1)
    

    for r in range(R):
        y[r] = model.addVar(name=f"y_{r}", vtype=vtype, lb=0, ub=1)

    
    for c in range(C):
        model.addCons(quicksum(x[(c, r, t)] 
                        for r in range(R)
                        for t in range(T)
                        if M[c, r] == 1) == 1)
    
    for r, t in product(range(R), range(T)):
        model.addCons(quicksum(x[(c, r, t)]
                        for c in range(C)
                        if M[c, r] == 1) <= 1)
    if optimal:
        for r in range(R):
            model.addCons(quicksum(x[(c, r, t)]
                            for c in range(C)
                            for t in range(T)
                            if M[c, r] == 1) <= (C*T + 1) * y[r])
        
    for t in range(T):
        for c1, c2 in L:
            model.addCons(quicksum(x[c1, r, t] for r in range(R) if M[c1, r] == 1) +
                          quicksum(x[c2, r, t] for r in range(R) if M[c2, r] == 1) 
                          <= 1)
    if optimal:
        model.setObjective(quicksum(y[r] for r in range(R)), sense="minimize")
    else:
        model.setObjective(0)

    model.optimize()
    return model, x

def print_schedule(model, x):
    res = {}
    for (c, r, t), var in x.items():
        if model.getVal(var) > 0.5:
            res[(r, t)] = c
        
    rows = sorted({r for r, _ in res.keys()})
    cols = sorted({t for _, t in res.keys()})

    table = []
    for r in rows:
        row = [f"{rooms[r][0]}\nCapacity = {rooms[r][1]}"]
        for t in cols:
            if res.get((r, t)) == None:
                row.append("")
            else:
                row.append(f"{courses[res.get((r, t))][0]}\nProf:{courses[res.get((r, t))][1]}\nEnrollment = {courses[res.get((r, t))][2]}")
        table.append(row)

    headers = [""] + list(time[cols])
    print(tabulate(table, headers=headers, tablefmt="grid"))


In [23]:
feasible_model, x = solve(courses, rooms, time, conflict_cap_array, conflict_instructor_list, optimal=False)

presolving:
(round 1, fast)       34 del vars, 100 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 547 clqs
   (0.0s) running MILP presolver
   (0.0s) MILP presolver found nothing
(round 2, exhaustive) 34 del vars, 100 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 547 upgd conss, 0 impls, 547 clqs
   (0.1s) probing: 51/7550 (0.7%) - 0 fixings, 0 aggregations, 0 implications, 0 bound changes
   (0.1s) probing aborted: 50/50 successive totally useless probings
   (0.1s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.2s) symmetry computation finished: 229 generators found (max: 1500, log10 of symmetry group size: 164.06) (symcode time: 0.16)
dynamic symmetry handling statistics:
   orbitopal reduction:       no components
   orbital reduction:         no components
   lexicographic reduction:   no permutations
handled 1 out of 1 symmetry components
(round 3, exhaustive) 260 del 

In [24]:
print_schedule(feasible_model, x)

+------------------+---------------------------+---------------------------+----------------------+--------------------+-------------------+--------------------------+---------------------+-------------------------+---------------------------+---------------------------+
|                  | 8                         | 9                         | 10                   | 11                 | 12                | 13                       | 14                  | 15                      | 16                        | 17                        |
+==================+===========================+===========================+======================+====================+===================+==========================+=====================+=========================+===========================+===========================+
| Wellman Hall 233 | MAT-115A - A01            | MAT-115A - A02            | MAT-115A - B01       | MAT-127A - A01     | MAT-127B - A01    | MAT-135A - B01           | MAT-135A - B02  

In [25]:
optimal_model, x = solve(courses, rooms, time, conflict_cap_array, conflict_instructor_list)

presolving:
(round 1, fast)       0 del vars, 110 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 24 chg coeffs, 0 upgd conss, 0 impls, 547 clqs
(round 2, fast)       10 del vars, 110 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 24 chg coeffs, 0 upgd conss, 0 impls, 547 clqs
   (0.0s) running MILP presolver
   (0.0s) MILP presolver found nothing
(round 3, exhaustive) 10 del vars, 110 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 24 chg coeffs, 571 upgd conss, 0 impls, 547 clqs
(round 4, medium)     10 del vars, 134 del conss, 755 add conss, 0 chg bounds, 0 chg sides, 24 chg coeffs, 571 upgd conss, 0 impls, 8851 clqs
   (0.1s) probing: 51/7574 (0.7%) - 0 fixings, 0 aggregations, 0 implications, 0 bound changes
   (0.1s) probing aborted: 50/50 successive totally useless probings
   (0.1s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.1s) symmetry computation finished: 58 generators found (max: 1500, log10 of symmetry gro

In [26]:
print(f"The number of room needed: {int(optimal_model.getObjVal())}")
print_schedule(optimal_model, x)


The number of room needed: 4
+------------------+---------------------------+--------------------------+---------------------------+--------------------------+----------------------+---------------------+---------------------------+---------------------------+----------------------+----------------------+
|                  | 8                         | 9                        | 10                        | 11                       | 12                   | 13                  | 14                        | 15                        | 16                   | 17                   |
+==================+===========================+==========================+===========================+==========================+======================+=====================+===========================+===========================+======================+======================+
| Wellman Hall 229 | MAT-115A - A02            | MAT-127A - A02           | MAT-115A - A01            | MAT-115A - B01           | MAT-13

In [27]:
relaxedLP_model, x = solve(courses, rooms, time, conflict_cap_array, conflict_instructor_list, ILP=False)

presolving:
(round 1, fast)       24 del vars, 134 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       34 del vars, 134 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
   (0.0s) running MILP presolver
   (0.0s) MILP presolver found nothing
   (0.1s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.2s) symmetry computation finished: 229 generators found (max: 1500, log10 of symmetry group size: 164.06) (symcode time: 0.16)
dynamic symmetry handling statistics:
   orbitopal reduction:       no components
   orbital reduction:         no components
   lexicographic reduction:   no permutations
handled 1 out of 1 symmetry components
(round 3, exhaustive) 34 del vars, 134 del conss, 5654 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (4 rounds: 4 fast, 2 medium, 2 exhaustive):
 34 dele

In [28]:
print(f"The optimal value for the objective function of the relaxed LP: {relaxedLP_model.getObjVal()}")

The optimal value for the objective function of the relaxed LP: 0.09973045822102428
